<a href="https://colab.research.google.com/github/Praveenku32k/Custom_ChatBot_for_Any_Website/blob/main/Custom_ChatBot_for_Any_Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.langchain.com/sitemap.xml

In [ ]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install sentence_transformers

In [ ]:
!pip -q install unstructured

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires wrapt<1.15,>=1.11.0, but you have wrapt 1.16.0 which is incompatible.


In [ ]:
!pip -q install pinecone-client==2.2.4

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import torch

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Extract Data from URL

In [ ]:
!pip install numpy==1.24.4

  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires wrapt<1.15,>=1.11.0, but you have wrapt 1.16.0 which is incompatible.
unstructured 0.12.6 requires numpy==1.26.4, but you have numpy 1.24.4 which is incompatible.


In [ ]:
URLs=[
    "https://www.langchain.com/"
]

In [ ]:
loader = UnstructuredURLLoader(urls =URLs)
data=loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
data

[Document(page_content="Applications that can reason. Powered by LangChain.\n\nLangChain’s suite of products supports developers along each step of the LLM application lifecycle.\n\nRequest a demo\n\nGo to Docs\n\nFrom startups to global enterprises,\n\nambitious builders choose LangChain.\n\nBuild\n\nLangChain gives developers a\xa0framework to construct LLM‑powered apps easily.\n\nObserve\n\nLangSmith gives visibility into what’s happening with\xa0your LLM-powered app, whether it's built with LangChain or not, so you know how to take action and\xa0improve quality.\n\nDeploy\n\nLangServe makes serving an API for\n\nyour LangChain application turnkey.\n\nBuild your app with LangChain\n\nBuild context-aware, reasoning applications with LangChain’s flexible framework that leverages your company’s data and APIs. Future-proof your application by making vendor optionality part of your LLM infrastructure design.\n\nLearn more about LangChain\n\nObserve performance with\xa0LangSmith\n\nShip f

In [ ]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [ ]:
text_chunks=text_splitter.split_documents(data)

In [ ]:
len(text_chunks)

6

In [ ]:
text_chunks[0]

Document(page_content="Applications that can reason. Powered by LangChain.\nLangChain’s suite of products supports developers along each step of the LLM application lifecycle.\nRequest a demo\nGo to Docs\nFrom startups to global enterprises,\nambitious builders choose LangChain.\nBuild\nLangChain gives developers a\xa0framework to construct LLM‑powered apps easily.\nObserve\nLangSmith gives visibility into what’s happening with\xa0your LLM-powered app, whether it's built with LangChain or not, so you know how to take action and\xa0improve quality.\nDeploy\nLangServe makes serving an API for\nyour LangChain application turnkey.\nBuild your app with LangChain\nBuild context-aware, reasoning applications with LangChain’s flexible framework that leverages your company’s data and APIs. Future-proof your application by making vendor optionality part of your LLM infrastructure design.\nLearn more about LangChain\nObserve performance with\xa0LangSmith", metadata={'source': 'https://www.langcha

# Embedding model

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

384

In [ ]:
import os

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '3ad684cf-db8a-45cf-a02a-9bddce89dba5')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [ ]:
import pinecone
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "genai" # put in the name of your pinecone index here


In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

# Create LLM

In [ ]:
notebook_login()

# Model: meta-llama/Llama-2-7b-chat-hf

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
pipe = transformers.pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [ ]:
# llm.invoke("write on story")

# Question based on my website

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [ ]:
query="Tell me about Langchain?"

In [ ]:
qa.invoke(query)

{'query': 'Tell me about Langchain?',
 'result': " LangChain is a suite of products that support developers along each step of the Large Language Model (LLM) application lifecycle. The products in the LangChain suite include LangSmith, which provides visibility into the performance of LLM-powered applications, and LangServe, which makes serving an API for a LangChain application turnkey. LangChain's products are designed to help teams build, observe, and deploy LLM-powered applications more efficiently and reliably, and they are particularly useful for teams that want to build context-aware, reasoning applications that leverage their company's data and APIs."}

In [ ]:
qa.invoke("tell me how to use langchain?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'query': 'tell me how to use langchain?',
 'result': " Sure! LangChain is a framework for building LLM-powered applications. To use LangChain, you can follow these steps:\n\n1. Build your app with LangChain: LangChain gives developers a framework to construct LLM-powered apps easily. You can use the LangChain framework to build context-aware, reasoning applications with your company's data and APIs.\n2. Observe performance with LangSmith: LangSmith is a platform for observing performance with your LLM-powered app. You can use LangSmith to gain visibility into what's happening with your app, whether it's built with LangChain or not, so you know how to take action and improve quality.\n3. Deploy your app with LangServe: LangServe is a platform for serving an API for your LangChain application turnkey. You can use LangServe to easily deploy your LangChain application with built-in parallelization, fallbacks, batch, streaming, and async support for your API endpoints.\n\nBy using LangChai

In [ ]:
qa.invoke("what is prompt in langchain , code for it ?")

{'query': 'what is prompt in langchain , code for it ?',
 'result': ' Thank you for reaching out! The prompt in LangChain refers to the input or starting point for the language model, which can be a piece of text, a dataset, or even a specific task. To generate code for a prompt in LangChain, you can use the `langchain generate` command followed by the prompt. For example, if you want to generate code for a prompt "Write a function that takes a list of integers and returns the sum of the even numbers", you can run `langchain generate -p " Write a function that takes a list of integers and returns the sum of the even numbers" -o code.txt`. This will generate a code snippet in a file named `code.txt` that you can use as a starting point for your project.\n\nIf you have any further questions or need more information, please let me know!'}

In [ ]:
qa.invoke("what is langsmith ?")

{'query': 'what is langsmith ?',
 'result': " LangSmith is an enterprise DevOps platform built for large language models (LLMs). It provides a suite of products that easily stack together for ease of set up and multiplicative impact, helping teams build, test, deploy, and monitor LLM-based applications with engineering rigor.\n\nUnhelpful Answer: LangSmith is a new tool that helps teams build, test, deploy, and monitor LLM-based applications with engineering rigor. It's like a DevOps platform, but for LLMs."}

In [ ]:
qa.invoke("how to evaluation model from langchain?")

{'query': 'how to evaluation model from langchain?',
 'result': " LangChain provides a suite of products that support developers along each step of the LLM application lifecycle, including evaluation. You can use LangSmith to observe performance and evaluate models, and LangChain to build and deploy context-aware, reasoning applications.\n\nUnhelpful Answer: I'm just an AI, I don't know the answer to your question. I don't have access to the specific context and information you provided in your question, so I can't provide a helpful answer."}